In [ ]:
import os

current_path = os.getcwd()

project_root = os.path.abspath(os.path.join(current_path, '..', '..'))

In [ ]:
import numpy as np

lbs100 = np.load(project_root+'/output/signal-unplanting/lbs100.npy')
lbs2000 = np.load(project_root+'/output/signal-unplanting/lbs2000.npy')
lbs10000 = np.load(project_root+'/output/signal-unplanting/lbs10000.npy')

S = np.load(project_root+'/output/signal-unplanting/S.npy')

lbsG = np.load(project_root+'/output/signal-unplanting/lbsG.npy')
lbsA = np.load(project_root+'/output/signal-unplanting/lbsA.npy')
lbsP = np.load(project_root+'/output/signal-unplanting/lbsP.npy')

n_values = np.load(project_root+'/output/signal-unplanting/n_values.npy')
n_valuesS = np.load(project_root+'/output/signal-unplanting/n_valuesS.npy')

lbsSP = np.array([lbsG,lbsA,lbsP])
lbsSU = np.array([lbs100,lbs2000,lbs10000])

In [ ]:
# pip install git+https://github.com/jhultman/matplotlib-curly-brace
from curlyBrace import curlyBrace

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tueplots import bundles, axes
from scipy.optimize import curve_fit
from tueplots import markers
from mpl_toolkits.axes_grid1.inset_locator import mark_inset # for zoom effect

%matplotlib widget

plt.rcParams.update(bundles.icml2022())
plt.rcParams.update(markers.inverted())
plt.rcParams.update(axes.grid())

colors = ['8c564b','d62728','9467bd','ff7f0e','1f77b4','2ca02c','17becf']

# labels for plot 0 and plot 1
labels0 = ['100', '2,000', '10,000']
labels1 = ['G', 'A', 'P']

scale_param = 10000 # used for fitting the sigmoid (adjust if needed)
x_values = n_values/scale_param
x_valuesb = n_valuesS/scale_param

# sigmoid used to fit
def sigmoid(x, a, b, c, d): 
    return d + c/(1.0 + np.exp(-a*(x-b)))

# bounds for sigmoid fitting (adjust if needed)
cmin = [-5,-1,-1,-1]
dmin = [-1,-0.02,-0.02,-0.02]
cmax = [10,20,6,6]
dmax = [0.01,15,1,1]
amin = [-2,-1,1,0]
bmin = [-2,-1,1,0]
amax = [2,1,1,0]
bmax = [2,1,1,0]

# do a grid with 2 rows to add space on top of the plots for adding zooms
fig, axs = plt.subplots(2, 3, figsize=(9, 4)) 
axes = [axs[1,0], axs[1,1], axs[1,2]]
axs[0,0].remove()
axs[0,1].remove()
# axs[0,2].remove()

# z-orders
zo = [1,3,2]

### plot 0: different values of n_{est}

for i in range(3): # 100 -> 2,000 -> 10,000
    ## sigmoid
    y_values = lbsSU[i]
    bds = ([min(y_values)+amin[i],min(x_values)+bmin[i],cmin[i],dmin[i]],[max(y_values)+amax[i],max(x_values)+bmax[i],cmax[i],dmax[i]])
    popt, pcov = curve_fit(sigmoid, x_values, y_values, method='dogbox', bounds=bds)
    x_fit = np.linspace(min(x_values), max(x_values))
    y_fit = np.minimum(sigmoid(x_fit, *popt),1)
    # artificially define a point (0,-0.015) for visual effect (sigmoid plot begin before 0)
    x_fit = np.concatenate((np.array([0]),x_fit))
    y_fit = np.concatenate((np.array([0]),y_fit))
    axes[0].plot(x_fit, y_fit,linewidth=3.5,color='#'+colors[i+3],label=labels0[i],zorder=zo[i])
    ## points
    axes[0].scatter(x_values,y_values,marker='o',edgecolor='#'+colors[i+3],color='white',linewidth=1,zorder=zo[i]+1,s=50)
    if i ==1: # for n_{est} = 2,000, plot it also on plot 1 and plot 2
        axes[1].plot(x_fit, y_fit,linewidth=3.5,zorder=4,color='#'+colors[i+3],label=labels0[i])
        axes[1].scatter(x_values,y_values,marker='o',edgecolor='#'+colors[i+3],color='white',linewidth=1,zorder=7,s=50)
        axes[2].plot(np.concatenate((np.array([0]),x_fit)), np.concatenate((np.array([0]),y_fit)),linewidth=3.5,zorder=4,color='#'+colors[i+3],label=labels0[i]) # concatenate with (0,0) to continue the plot before the computed values
        axes[2].scatter(x_values,y_values,marker='o',edgecolor='#'+colors[i+3],color='white',linewidth=1,zorder=4,s=50)

axes[0].legend(loc='lower right', fontsize=9)
axes[0].set_xlim(0,16)

### plot 1: comparison between naive and adaptive strategies

# z-orders
zo1 = [3,1,2]

for i in range(3): # Good -> Average -> Poor
    ## sigmoid
    y_values = lbsSP[i]
    bds = ([min(y_values)+amin[i],min(x_values)+bmin[i],cmin[i],dmin[i]],[max(y_values)+amax[i],max(x_values)+bmax[i],cmax[i],dmax[i]])
    popt, pcov = curve_fit(sigmoid, x_values, y_values, method='dogbox', bounds=bds)
    x_fit = np.linspace(min(x_values), max(x_values))
    y_fit = np.minimum(sigmoid(x_fit, *popt),1)
    # since Average and Poor overlap, draw one with dashed lines
    axes[1].plot(x_fit, y_fit,linewidth=3.5,zorder=zo1[i],color='#'+colors[i],label=labels1[i])
    ## points
    axes[1].scatter(x_values,y_values,marker='o',edgecolor='#'+colors[i],color='white',linewidth=1,zorder=zo1[i]+3,s=50)

axes[1].legend(loc='lower right', fontsize=9)
axes[1].set_xlim(0,20)

### plot 2: comparison between adaptive strategy and true success

# add true success (n_{est} = 2000 already plotted)
## sigmoid
y_values = S
popt, pcov = curve_fit(sigmoid, x_valuesb, y_values, method='dogbox', bounds=([min(y_values)-1,min(x_valuesb)-1,-1,-1],[max(y_values)+2,max(x_valuesb)+2,5,0.01]))
x_fit = np.linspace(min(x_valuesb), max(x_valuesb))
y_fit = np.minimum(sigmoid(x_fit, *popt),1)
axes[2].plot(x_fit, y_fit,linewidth=3.5,zorder=1,color='#'+colors[6],label=r'$\hat{S}(n)$',linestyle=(0,(1,1)))
## points
axes[2].scatter(x_valuesb,y_values,marker='o',edgecolor='#'+colors[6],color='white',linewidth=1,zorder=2,s=50)

axes[2].legend(loc='lower right', fontsize=9)
axes[2].set_xlim(0,18)
    
# adjust tick labels
axes[0].set_xticks(np.arange(0, 30 + 1, 1), minor=True) 
axes[0].set_xticks(np.arange(0, 30 + 1, 5), minor=False) 
axes[0].set_xticklabels(np.arange(0, 30 + 1, 5), minor=False)
axes[1].set_xticks(np.arange(0, 30 + 1, 1), minor=True) 
axes[1].set_xticks(np.arange(0, 30 + 1, 5), minor=False) 
axes[1].set_xticklabels(np.arange(0, 30 + 1, 5), minor=False)
axes[2].set_xticks(np.arange(0, 30 + 1, 1), minor=True) 
axes[2].set_xticks(np.arange(0, 30 + 1, 5), minor=False) 
axes[2].set_xticklabels(np.arange(0, 30 + 1, 5), minor=False)
for i in range(3):
    axes[i].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: int(x))) 
    axes[i].tick_params(axis='x', which='major', length=7)
    axes[i].tick_params(axis='x', which='minor', length=4) 
    axes[i].tick_params(axis='x', which='both', width=1) 
    axes[i].tick_params(axis='both', labelsize=10)
    axes[i].grid()
    axes[i].set_ylim(-0.05,1.05)
    axes[i].set_yticks(np.arange(0, 101, 5)/100, minor=True) 
    axes[i].set_yticks(np.arange(0, 101, 20)/100, minor=False) 
    axes[i].set_yticklabels(np.arange(0, 101, 20)/100, minor=False)
    axes[i].tick_params(axis='y', which='major', length=7)
    axes[i].tick_params(axis='y', which='minor', length=4) 
    axes[i].tick_params(axis='y', which='both', width=1) 

fig.supylabel('Success', fontsize=12, x=0.01, y=0.46)                                     
fig.supxlabel(r'Relative collective size $n/N$ (in $\%$)', fontsize=12, y=0.1)               

axes[0].text(0.5, -0.4, "(a)", transform=axes[0].transAxes, ha='center', fontsize=12)
axes[1].text(0.5, -0.4, "(b)", transform=axes[1].transAxes, ha='center', fontsize=12)
axes[2].text(0.5, -0.4, "(c)", transform=axes[2].transAxes, ha='center', fontsize=12)

# force spines to be drawn with higher z-order so that points are not on top of axes spines
for ax in axes: 
    for spine in ax.spines.values():
        spine.set_zorder(10) 

plt.tight_layout()
plt.subplots_adjust(bottom=0.3, wspace=0.3) 

### zoom effects

# inset axes plot 1
x1, x2, y1, y2 = 4, 16, -0.05, 1.05  # subregion of the original image
axins1 = axes[1].inset_axes(
    [0, 1.3, 1, 0.7], # lower left coordinate + width + height            
    xlim=(x1, x2), ylim=(y1, y2), xticklabels=[], yticklabels=[])
axins1.set_xticks(np.arange(4, 16 + 1, 1), minor=True) 
axins1.set_xticks(np.arange(4, 16 + 1, 2), minor=False) 
axins1.set_xticklabels(np.arange(4, 16 + 1, 2), minor=False)
axins1.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: int(x))) 
axins1.tick_params(axis='x', which='major', length=7)
axins1.tick_params(axis='x', which='minor', length=4) 
axins1.tick_params(axis='x', which='both', width=1) 
axins1.tick_params(axis='both', labelsize=10)
axins1.grid()
axins1.set_ylim(-0.05,1.05)
axins1.set_yticks(np.arange(0, 101, 5)/100, minor=True) 
axins1.set_yticks(np.arange(0, 101, 20)/100, minor=False) 
axins1.set_yticklabels(np.arange(0, 101, 20)/100, minor=False)
axins1.tick_params(axis='y', which='major', length=7)
axins1.tick_params(axis='y', which='minor', length=4) 
axins1.tick_params(axis='y', which='both', width=1) 
_,pp1,pp2 = mark_inset(axes[1], axins1, loc1=3, loc2=3, ec="grey",ls=':',lw=2)
pp1.loc1 = 3
pp1.loc2 = 2
pp2.loc1 = 4
pp2.loc2 = 1
for i in range(3): # Good -> Average -> Poor
    ## sigmoid
    y_values = lbsSP[i]
    bds = ([min(y_values)+amin[i],min(x_values)+bmin[i],cmin[i],dmin[i]],[max(y_values)+amax[i],max(x_values)+bmax[i],cmax[i],dmax[i]])
    popt, pcov = curve_fit(sigmoid, x_values, y_values, method='dogbox', bounds=bds)
    x_fit = np.linspace(min(x_values), max(x_values))
    y_fit = np.minimum(sigmoid(x_fit, *popt),1)
    axins1.plot(x_fit, y_fit,linewidth=3.5,zorder=zo1[i],color='#'+colors[i],label=labels1[i])
    ## points
    axins1.scatter(x_values,y_values,marker='o',edgecolor='#'+colors[i],color='white',linewidth=1,zorder=zo1[i]+3,s=50)
for spine in axins1.spines.values():
        spine.set_zorder(10) 

# inset axes plot 0
x1, x2, y1, y2 = 4, 16, -0.05, 1.05  # subregion of the original image
axins = axes[0].inset_axes(
    [0, 1.3, 1, 0.7], # lower left coordinate + width + height
    xlim=(x1, x2), ylim=(y1, y2), xticklabels=[], yticklabels=[])
axins.set_xticks(np.arange(4, 16 + 1, 1), minor=True) 
axins.set_xticks(np.arange(4, 16 + 1, 2), minor=False) 
axins.set_xticklabels(np.arange(4, 16 + 1, 2), minor=False)
axins.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: int(x))) 
axins.tick_params(axis='x', which='major', length=7)
axins.tick_params(axis='x', which='minor', length=4) 
axins.tick_params(axis='x', which='both', width=1) 
axins.tick_params(axis='both', labelsize=10)
axins.grid()
axins.set_ylim(-0.05,1.05)
axins.set_yticks(np.arange(0, 101, 5)/100, minor=True) 
axins.set_yticks(np.arange(0, 101, 20)/100, minor=False) 
axins.set_yticklabels(np.arange(0, 101, 20)/100, minor=False)
axins.tick_params(axis='y', which='major', length=7)
axins.tick_params(axis='y', which='minor', length=4) 
axins.tick_params(axis='y', which='both', width=1) 
_,pp1,pp2 = mark_inset(axes[0], axins, loc1=3, loc2=3, ec="grey",ls=':',lw=2)
pp1.loc1 = 3
pp1.loc2 = 2
pp2.loc1 = 4
pp2.loc2 = 1
for i in range(3): # 100 -> 2,000 -> 10,000
    ## sigmoid
    y_values = lbsSU[i]
    bds = ([min(y_values)+amin[i],min(x_values)+bmin[i],cmin[i],dmin[i]],[max(y_values)+amax[i],max(x_values)+bmax[i],cmax[i],dmax[i]])
    popt, pcov = curve_fit(sigmoid, x_values, y_values, method='dogbox', bounds=bds)
    x_fit = np.linspace(min(x_values), max(x_values))
    y_fit = np.minimum(sigmoid(x_fit, *popt),1)
    # artificially define a point (0,-0.015) for visual effect (sigmoid plot begin before 0)
    x_fit = np.concatenate((np.array([0]),x_fit))
    y_fit = np.concatenate((np.array([0]),y_fit))
    axins.plot(x_fit, y_fit,linewidth=3.5,color='#'+colors[i+3],label=labels0[i],zorder=zo[i])
    ## points
    axins.scatter(x_values,y_values,marker='o',edgecolor='#'+colors[i+3],color='white',linewidth=1,zorder=zo[i]+1,s=50)
    if i==1:
        axins1.plot(x_fit, y_fit,linewidth=3.5,zorder=4,color='#'+colors[i+3],label=labels0[i])
        axins1.scatter(x_values,y_values,marker='o',edgecolor='#'+colors[i+3],color='white',linewidth=1,zorder=7,s=50)
for spine in axins.spines.values():
        spine.set_zorder(10) 

# bracket and text
axs[0,2].set_xlim(0,10)
axs[0,2].set_ylim(0,10)
axs[0,2].spines['top'].set_visible(False)
axs[0,2].spines['right'].set_visible(False)
axs[0,2].spines['left'].set_visible(False)
axs[0,2].spines['bottom'].set_visible(False)
axs[0,2].set_xticks([])
axs[0,2].set_yticks([])
curlyBrace(fig, axs[0,2], [0,7.5], [0,0.1], 0.12, bool_auto=True, str_text='', color='black', lw=2)
fig.text(0.84, 0.755, 'Zoomed-in view', ha='center', va='bottom', fontsize=12)

plt.savefig(project_root+"/plots/signal-unplanting.pdf", format="pdf", bbox_inches="tight")

plt.show()